In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('../..')

In [6]:
import deepmatcher as dm
import pandas as pd
from utils.dataset_parser import generate_unlabeled,dropTokensInColumns
import itertools
from utils.deepmatcher_utils import wrapDm

In [6]:
DATA_DIR = '../../datasets/Structured/Beer/'
tableA = pd.read_csv(DATA_DIR+'tableA.csv')
tableB = pd.read_csv(DATA_DIR+'tableB.csv')

In [5]:
def containTokens(attribute,tokens):
    attr_tokens = set(map(lambda t:t.lower(),attribute.split()))
    return set(tokens).issubset(attr_tokens)

In [6]:
def filterTable(table,attribute,tokens):
    filter_row = list(map(lambda attr:containTokens(attr,tokens),table[attribute].values))
    return table[filter_row].copy()

In [7]:
filteredTableA = filterTable(tableA,'Beer_Name',['imperial','red','ale'])
filteredTableB = filterTable(tableB,'Beer_Name',['imperial','red','ale'])
len(filteredTableA),len(filteredTableB)

(58, 47)

In [8]:
unlabeled_ids = list(itertools.product(filteredTableA.id.values,filteredTableB.id.values))
unlabeled_df = pd.DataFrame(unlabeled_ids,columns=['ltable_id','rtable_id'])
unlabeled_df.to_csv(DATA_DIR+'unlabeled_ids.csv',index=False)

In [9]:
unlabeled_values = generate_unlabeled(DATA_DIR,'unlabeled_ids.csv')

In [10]:
unlabeled_values.to_csv(DATA_DIR+'unlabeled.csv',index=False)

In [7]:
unlabeled_df = pd.read_csv(DATA_DIR+'unlabeled.csv')

In [4]:
model_hybrid = dm.MatchingModel(attr_summarizer='hybrid')
model_hybrid.load_state('../../models/beer_hybrid.pth')
model_hybrid = model_hybrid.to('cuda')

In [75]:
model_rnn = dm.MatchingModel(attr_summarizer='rnn')
model_rnn.load_state('../../models/beer_rnn.pth')
model_rnn = model_hybrid.to('cuda')

In [9]:
prediction_rnn = wrapDm(unlabeled_df,model_hybrid,outputAttributes=True)

In [19]:
prediction_rnn[(prediction_rnn.match_score>=0.3)]

,match_score,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV
id,,,,,,,,,
101,0.350810,Imperial Moustache Red Ale,Salty Nut Brewery,American Amber / Red Ale,6.00 %,Marble Imperial Red Ale,Marble Brewery,American Strong Ale,9 %
174,0.430955,Lagunitas Imperial Red Ale,Lagunitas Brewing Company,American Amber / Red Ale,7.80 %,Lagunitas Imperial Red Ale,Lagunitas Brewing Company &#40; Heineken &#41;,American Strong Ale,7.80 %
375,0.344118,Deranger Imperial Red Ale,Laurelwood Public House & Brewery,American Amber / Red Ale,8.60 %,Laurelwood Organic Deranger Imperial Red Ale,Laurelwood Public House & Brewery,American Strong Ale,8.60 %
1140,0.325326,Tst Brw 01 - Imperial Red Ale,Brouwerij De Koninck NV,American Amber / Red Ale,7.80 %,De Koninck TSTBRW 01 Imperial Red Ale,Brouwerij De Koninck &#40; Duvel-Moortgat &#41;,American Strong Ale,7.80 %
1517,0.433033,Lavery Imperial Red Ale,Lavery Brewing Company,American Amber / Red Ale,8.20 %,Lavery Imperial Red Ale &#40; IRA &#41;,Lavery Brewing Company,American Strong Ale,8.20 %
1666,0.312181,Ale Mania Imperial Red Ale,Ale-Mania,American Amber / Red Ale,9.50 %,Ale Mania Imperial Red Ale,Ale Mania &#40; Fritz Ale &#41;,American Strong Ale,9.50 %
1843,0.308932,Imperial Red Ale ( Spring Seasonal ),Edge Brewing Company,American Amber / Red Ale,8.50 %,Edge Imperial Red Ale &#40; Spring Seasonal &#41;,Edge Brewing Company,American Strong Ale,8.50 %


## Load improved model

In [21]:
improved_mod = dm.MatchingModel(attr_summarizer='hybrid')
improved_mod.load_state('../../models/beer_improved_hybrid.pth')
improved_mod = improved_mod.to('cuda')

In [22]:
prediction_improved = wrapDm(unlabeled_df,improved_mod,outputAttributes=True)

In [28]:
possible_matching = prediction_improved[(prediction_improved.match_score>=0.5)].copy()
possible_matching['label'] = [1]*len(possible_matching)
len(possible_matching)

21

In [29]:
possible_matching

,match_score,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV,label
id,,,,,,,,,,
174,0.983587,Lagunitas Imperial Red Ale,Lagunitas Brewing Company,American Amber / Red Ale,7.80 %,Lagunitas Imperial Red Ale,Lagunitas Brewing Company &#40; Heineken &#41;,American Strong Ale,7.80 %,1
306,0.976538,Double Dragon Imperial Red Ale,Phillips Brewing Company,American Amber / Red Ale,8.20 %,Phillips Double Dragon Imperial Red Ale,Phillips Brewing Co. .,American Strong Ale,8.20 %,1
375,0.940306,Deranger Imperial Red Ale,Laurelwood Public House & Brewery,American Amber / Red Ale,8.60 %,Laurelwood Organic Deranger Imperial Red Ale,Laurelwood Public House & Brewery,American Strong Ale,8.60 %,1
409,0.772146,Alaskan Imperial Red Ale ( Pilot Series ),Alaskan Brewing Co. .,American Amber / Red Ale,8.50 %,Yukon You Only Live Once Imperial Red Ale,Yukon Brewing Company,Amber Ale,7.50 %,1
752,0.524482,Democratic Imperial Red Ale,BrewDog,American Amber / Red Ale,9.00 %,406 Imperial Red Ale,406 Brewing Company,American Strong Ale,9.20 %,1
785,0.958552,Stupiphany Imperial Red Ale,Rusty Truck Brewing Company / Roadhouse 101,American Amber / Red Ale,8.00 %,Rusty Truck Stupiphany Imperial Red Ale,Rusty Truck Brewing / Roadhouse 101,American Strong Ale,8 %,1
836,0.660879,Naughty Redhead Imperial Red Ale,Sick N Twisted Brewing Co. .,American Amber / Red Ale,8.00 %,Sick N Twisted Naughty Redhead Imperial Red Ale,Sick N Twisted Brewing Company,Amber Ale,8 %,1
876,0.973856,6th Anniversary Ale ( Dirty Hippie Imperial Re...,Kern River Brewing Company,American Amber / Red Ale,8.00 %,Kern River Dirty Hippie Imperial Red Ale,Kern River Brewing,American Strong Ale,8 %,1
914,0.569625,Red 88 Imperial Amber Ale,River City Brewing Co. .,American Amber / Red Ale,8.00 %,Kern River Dirty Hippie Imperial Red Ale,Kern River Brewing,American Strong Ale,8 %,1


In [27]:
possible_matching.to_csv('possibleMatchings2.csv',index=False)

## Load improved model 2

In [8]:
improved_mod2 = dm.MatchingModel(attr_summarizer='hybrid')
improved_mod2.load_state('../../models/beer_improved_hybrid2.pth')
improved_mod2 = improved_mod2.to('cuda')

In [9]:
unlabeled_df = pd.read_csv('../../datasets/Structured/Beer/unlabeled.csv')

In [11]:
prediction_improved = wrapDm(unlabeled_df,improved_mod2,outputAttributes=True)

In [18]:
possible_matching = prediction_improved[(prediction_improved.match_score>=0.5)].copy()
len(possible_matching)

18

In [19]:
possible_matching

,match_score,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV
id,,,,,,,,,
174,0.992928,Lagunitas Imperial Red Ale,Lagunitas Brewing Company,American Amber / Red Ale,7.80 %,Lagunitas Imperial Red Ale,Lagunitas Brewing Company &#40; Heineken &#41;,American Strong Ale,7.80 %
306,0.972071,Double Dragon Imperial Red Ale,Phillips Brewing Company,American Amber / Red Ale,8.20 %,Phillips Double Dragon Imperial Red Ale,Phillips Brewing Co. .,American Strong Ale,8.20 %
375,0.919797,Deranger Imperial Red Ale,Laurelwood Public House & Brewery,American Amber / Red Ale,8.60 %,Laurelwood Organic Deranger Imperial Red Ale,Laurelwood Public House & Brewery,American Strong Ale,8.60 %
409,0.646307,Alaskan Imperial Red Ale ( Pilot Series ),Alaskan Brewing Co. .,American Amber / Red Ale,8.50 %,Yukon You Only Live Once Imperial Red Ale,Yukon Brewing Company,Amber Ale,7.50 %
785,0.945212,Stupiphany Imperial Red Ale,Rusty Truck Brewing Company / Roadhouse 101,American Amber / Red Ale,8.00 %,Rusty Truck Stupiphany Imperial Red Ale,Rusty Truck Brewing / Roadhouse 101,American Strong Ale,8 %
836,0.982906,Naughty Redhead Imperial Red Ale,Sick N Twisted Brewing Co. .,American Amber / Red Ale,8.00 %,Sick N Twisted Naughty Redhead Imperial Red Ale,Sick N Twisted Brewing Company,Amber Ale,8 %
876,0.966113,6th Anniversary Ale ( Dirty Hippie Imperial Re...,Kern River Brewing Company,American Amber / Red Ale,8.00 %,Kern River Dirty Hippie Imperial Red Ale,Kern River Brewing,American Strong Ale,8 %
1277,0.988235,Tongue Buckler - Imperial Red Ale,Ballast Point Brewing Company,American Amber / Red Ale,10.00 %,Ballast Point Tongue Buckler Imperial Red Ale ...,Ballast Point Brewing Company,American Strong Ale,10 %
1517,0.991773,Lavery Imperial Red Ale,Lavery Brewing Company,American Amber / Red Ale,8.20 %,Lavery Imperial Red Ale &#40; IRA &#41;,Lavery Brewing Company,American Strong Ale,8.20 %
